In [66]:
%pip install numpy
%pip install sympy
%pip install scipy
%load_ext autoreload
%autoreload 2
from simplex_two_phases import simplex as simplex_two_phases, _simplex_find_feasible_initial_basis
from simplex_tableau import simplex as simplex_tableau
#from simplex_tableau import markdown_repr_T
from simplex import simplex
from revised_simplex_tableau import revised_simplex_tableau, markdown_repr_T
from utils import array_to_markdown

    
import numpy as np
from pprint import pprint
from IPython.display import Markdown, display
from itertools import combinations
from sympy import symbols, Matrix, solve
from scipy.optimize import linprog


solution_types = {
    -1: 'Unfeasible',
    1: 'Optimal finite solution found',
    2: 'Multiple optimal solutions found',
    3: 'Unbounded'
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Exercicio 4.1

Verificando se as restrições de não negatividade para $ {A_{I}}^{-1} $ são violadas

In [67]:
A = np.array([
    [ 1, 1, -1, 0, 0],
    [-2, 3,  0, 1, 0],
    [ 0, 1,  0, 0, 1]
])

B = np.array([
    1,
    6,
    2
])

C = np.array([1, -3, 0, 0, 0])

I = [2, 3, 4]

A_I = A[:,I] # definição da matriz básica
A_I_inv = np.linalg.inv(A_I)

result = f'Restrições de negatividade{
    ' não ' if np.all(A_I_inv @ B >= 0) else ' '
}violadas'

display(
    Markdown(
        result
    )
)

Restrições de negatividade violadas

Como as restrições de negatividade foram violadas, devemos criar um problema artificial e resolvê-lo para:

1. Verificar a factibilidade do problema original
2. Encontrar uma base inicial factível para o problema original que seja capaz de resolvê-lo

O problema artificial é do tipo:

$$
\begin{aligned}
    & \text{min} \quad \phi = & \textbf{1} \: x_a \\
    & \text{s. t.} & A x + x_a &= b \\
    &              &       x   &\geq 0 \\
    &              &       x_a &\geq 0 \\
\end{aligned}
$$

Vamos resolvê-lo abaixo:

In [68]:

m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = [5, 3, 4]

# resolvendo problema artificial com o simplex tableau revisado
solution = revised_simplex_tableau(A_artificial, B, C_artificial, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | 1.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{5}\\) | 1.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{3}\\) | 0.000 | 1.000 | 0.000 | 6.000 |
| \\(X_{4}\\) | 0.000 | 0.000 | 1.000 | 2.000 |


Extended Tableau

|   |  | \\(A_I^{-1}\\) |   | RHS | \\(X_{0}\\)
|---|---|---|---|---|---|
| Z | 1.000 | 0.000 | 0.000 | 1.000 | 1.000 |
| \\(X_{5}\\) | 1.000 | 0.000 | 0.000 | 1.000 | 1.000 |
| \\(X_{3}\\) | 0.000 | 1.000 | 0.000 | 6.000 | -2.000 |
| \\(X_{4}\\) | 0.000 | 0.000 | 1.000 | 2.000 | 0.000 |


\\(Ĉ_J = \ \\)array([ 1.,  1., -1.])


\\(J = {[0, 1, 2]}\\)

### Pivot Operations

Variable to enter: \\(X_{0}\\)
Variable to leave: \\(X_{5}\\)
#### Row Operations:

1. \\(R_{X_{5}} \leftarrow \frac{R_{X_{5}}}{1.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 1.000R_{X_{5}}\\)
2. \\(R_{X_{3}} \leftarrow R_{X_{3}} + 2.000R_{X_{5}}\\)
3. \\(R_{X_{4}} \leftarrow R_{X_{4}} - 0.000R_{X_{5}}\\)


Computed Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{0}\\) | 1.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{3}\\) | 2.000 | 1.000 | 0.000 | 8.000 |
| \\(X_{4}\\) | 0.000 | 0.000 | 1.000 | 2.000 |


## Iteration 1

Starting Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{0}\\) | 1.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{3}\\) | 2.000 | 1.000 | 0.000 | 8.000 |
| \\(X_{4}\\) | 0.000 | 0.000 | 1.000 | 2.000 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
1.000 & 1.000 & -1.000 & 0.000 & 0.000 & 1.000 \\
-2.000 & 3.000 & 0.000 & 1.000 & 0.000 & 0.000 \\
0.000 & 1.000 & 0.000 & 0.000 & 1.000 & 0.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
1.000\\
6.000\\
2.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal multiple solutions

Optimal Solution: \\(X^{*} = ['1.000', '0.000', '0.000', '8.000', '2.000', '0.000']\\)

Optimal Value: \\(Z^{*} = 0.000\\)

Optimal Basis: \\([0, 3, 4]\\)

Final non-basic variables set J: \\([5, 1, 2]\\)

\\(\hat{C}_{J}\\): \\([-1.0, 0.0, 0.0]\\)

Number of Iterations: 2



Verificamos que $ \phi^{*} = 0 $, o que caracteriza o problema original como factível.

Portanto, vamos resolvê-lo à partir da base inicial facítvel obtida na resolução do artificial $ I = [0, 3, 4] $

In [69]:
I = [0, 3, 4]

# resolvendo problema original com o simplex tableau revisado
solution = revised_simplex_tableau(A, B, C, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | 1.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{0}\\) | 1.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{3}\\) | 2.000 | 1.000 | 0.000 | 8.000 |
| \\(X_{4}\\) | 0.000 | 0.000 | 1.000 | 2.000 |


Extended Tableau

|   |  | \\(A_I^{-1}\\) |   | RHS | \\(X_{1}\\)
|---|---|---|---|---|---|
| Z | 1.000 | 0.000 | 0.000 | 1.000 | 4.000 |
| \\(X_{0}\\) | 1.000 | 0.000 | 0.000 | 1.000 | 1.000 |
| \\(X_{3}\\) | 2.000 | 1.000 | 0.000 | 8.000 | 5.000 |
| \\(X_{4}\\) | 0.000 | 0.000 | 1.000 | 2.000 | 1.000 |


\\(Ĉ_J = \ \\)array([ 4., -1.])


\\(J = {[1, 2]}\\)

### Pivot Operations

Variable to enter: \\(X_{1}\\)
Variable to leave: \\(X_{0}\\)
#### Row Operations:

1. \\(R_{X_{0}} \leftarrow \frac{R_{X_{0}}}{1.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 4.000R_{X_{0}}\\)
2. \\(R_{X_{3}} \leftarrow R_{X_{3}} - 5.000R_{X_{0}}\\)
3. \\(R_{X_{4}} \leftarrow R_{X_{4}} - 1.000R_{X_{0}}\\)


Computed Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | -3.000 | 0.000 | 0.000 | -3.000 |
| \\(X_{1}\\) | 1.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{3}\\) | -3.000 | 1.000 | 0.000 | 3.000 |
| \\(X_{4}\\) | -1.000 | 0.000 | 1.000 | 1.000 |


## Iteration 1

Starting Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | -3.000 | 0.000 | 0.000 | -3.000 |
| \\(X_{1}\\) | 1.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{3}\\) | -3.000 | 1.000 | 0.000 | 3.000 |
| \\(X_{4}\\) | -1.000 | 0.000 | 1.000 | 1.000 |


Extended Tableau

|   |  | \\(A_I^{-1}\\) |   | RHS | \\(X_{2}\\)
|---|---|---|---|---|---|
| Z | -3.000 | 0.000 | 0.000 | -3.000 | 3.000 |
| \\(X_{1}\\) | 1.000 | 0.000 | 0.000 | 1.000 | -1.000 |
| \\(X_{3}\\) | -3.000 | 1.000 | 0.000 | 3.000 | 3.000 |
| \\(X_{4}\\) | -1.000 | 0.000 | 1.000 | 1.000 | 1.000 |


\\(Ĉ_J = \ \\)array([-4.,  3.])


\\(J = {[0, 2]}\\)

### Pivot Operations

Variable to enter: \\(X_{2}\\)
Variable to leave: \\(X_{3}\\)
#### Row Operations:

1. \\(R_{X_{3}} \leftarrow \frac{R_{X_{3}}}{3.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 3.000R_{X_{3}}\\)
2. \\(R_{X_{1}} \leftarrow R_{X_{1}} + 1.000R_{X_{3}}\\)
3. \\(R_{X_{4}} \leftarrow R_{X_{4}} - 1.000R_{X_{3}}\\)


Computed Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | 0.000 | -1.000 | 0.000 | -6.000 |
| \\(X_{1}\\) | 0.000 | 0.333 | 0.000 | 2.000 |
| \\(X_{2}\\) | -1.000 | 0.333 | 0.000 | 1.000 |
| \\(X_{4}\\) | 0.000 | -0.333 | 1.000 | 0.000 |


## Iteration 2

Starting Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | 0.000 | -1.000 | 0.000 | -6.000 |
| \\(X_{1}\\) | 0.000 | 0.333 | 0.000 | 2.000 |
| \\(X_{2}\\) | -1.000 | 0.333 | 0.000 | 1.000 |
| \\(X_{4}\\) | 0.000 | -0.333 | 1.000 | 0.000 |


Extended Tableau

|   |  | \\(A_I^{-1}\\) |   | RHS | \\(X_{0}\\)
|---|---|---|---|---|---|
| Z | 0.000 | -1.000 | 0.000 | -6.000 | 1.000 |
| \\(X_{1}\\) | 0.000 | 0.333 | 0.000 | 2.000 | -0.667 |
| \\(X_{2}\\) | -1.000 | 0.333 | 0.000 | 1.000 | -1.667 |
| \\(X_{4}\\) | 0.000 | -0.333 | 1.000 | 0.000 | 0.667 |


\\(Ĉ_J = \ \\)array([ 1., -1.])


\\(J = {[0, 3]}\\)

### Pivot Operations

Variable to enter: \\(X_{0}\\)
Variable to leave: \\(X_{4}\\)
#### Row Operations:

1. \\(R_{X_{4}} \leftarrow \frac{R_{X_{4}}}{0.667}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 1.000R_{X_{4}}\\)
2. \\(R_{X_{1}} \leftarrow R_{X_{1}} + 0.667R_{X_{4}}\\)
3. \\(R_{X_{2}} \leftarrow R_{X_{2}} + 1.667R_{X_{4}}\\)


Computed Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | 0.000 | -0.500 | -1.500 | -6.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 1.000 | 2.000 |
| \\(X_{2}\\) | -1.000 | -0.500 | 2.500 | 1.000 |
| \\(X_{0}\\) | 0.000 | -0.500 | 1.500 | 0.000 |


## Iteration 3

Starting Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | 0.000 | -0.500 | -1.500 | -6.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 1.000 | 2.000 |
| \\(X_{2}\\) | -1.000 | -0.500 | 2.500 | 1.000 |
| \\(X_{0}\\) | 0.000 | -0.500 | 1.500 | 0.000 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
1.000 & 1.000 & -1.000 & 0.000 & 0.000 \\
-2.000 & 3.000 & 0.000 & 1.000 & 0.000 \\
0.000 & 1.000 & 0.000 & 0.000 & 1.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
1.000\\
6.000\\
2.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
1.000 & -3.000 & 0.000 & 0.000 & 0.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal unique solution

Optimal Solution: \\(X^{*} = ['0.000', '2.000', '1.000', '0.000', '0.000']\\)

Optimal Value: \\(Z^{*} = -6.000\\)

Optimal Basis: \\([1, 2, 0]\\)

Final non-basic variables set J: \\([4, 3]\\)

\\(\hat{C}_{J}\\): \\([-1.5, -0.5]\\)

Number of Iterations: 4



## Result Comparison through scipy

In [70]:
result = linprog(C, A_eq = A, b_eq = B, method='highs')
print('Result comparison, we found the optimal solution through our tableaus')
display(result)

Result comparison, we found the optimal solution through our tableaus


        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -6.0
              x: [ 0.000e+00  2.000e+00  1.000e+00 -0.000e+00  0.000e+00]
            nit: 0
          lower:  residual: [ 0.000e+00  2.000e+00  1.000e+00 -0.000e+00
                              0.000e+00]
                 marginals: [ 1.000e+00  0.000e+00  0.000e+00  0.000e+00
                              3.000e+00]
          upper:  residual: [       inf        inf        inf        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00]
          eqlin:  residual: [ 0.000e+00  0.000e+00  0.000e+00]
                 marginals: [-0.000e+00 -0.000e+00 -3.000e+00]
        ineqlin:  residual: []
                 marginals: []
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

## Result Comparison through Simplex Two Phases (without tableaus)

In [71]:
I = [2, 3, 4]
_, n = A.shape
X = np.zeros(n)
z_star, x_star, I_star, A_I_star, A, iterations_count, solution_type, debug_info  = simplex_two_phases(A, B, C, I, debug=True)
X[I_star] = x_star

result_md = f'''
Solution Type: {solution_types[solution_type]}

$ Z^{{*}} = {z_star} $

$ X^{{*}} = \\begin{{bmatrix}}
    { '\n'.join([f'{x:.2f} \\\\' for x in X]) }
\\end{{bmatrix}}
$

$ I^{{*}} = $ {I_star.tolist()}

'''
display(
    Markdown(
        result_md
    )
)


Solution Type: Optimal finite solution found

$ Z^{*} = -6.0 $

$ X^{*} = \begin{bmatrix}
    0.00 \\
2.00 \\
1.00 \\
0.00 \\
0.00 \\
\end{bmatrix}
$

$ I^{*} = $ [1, 2, 0]

